In [1]:
import mlflow
import pandas as pd
from mlflow import MlflowClient

In [38]:
REMOTE_TRACKING_URI = "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
experiment_name = "small_data"
mlflow.set_tracking_uri(REMOTE_TRACKING_URI)
client = MlflowClient(REMOTE_TRACKING_URI)
dataset = "lenta"

experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_id=experiment.experiment_id)

<Experiment: artifact_location='s3://kaggle-nakayama/mlflow/7', creation_time=1731298184492, experiment_id='7', last_update_time=1731298184492, lifecycle_stage='active', name='small_data', tags={}>

In [39]:
runs = mlflow.search_runs(experiment_ids=experiment.experiment_id)
name_flg = runs["tags.mlflow.runName"].str.contains(dataset)
run_df = runs.loc[name_flg]

In [40]:
run_df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.system/disk_usage_megabytes,metrics.qini_at_10,metrics.system/cpu_utilization_percentage,metrics.system/network_receive_megabytes,...,tags.dataset,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.parentRunId,tags.package,tags.model,tags.mlflow.source.git.commit,tags.mlflow.note.content
6,62eccbddb74f499a8be313e8df18d603,7,FINISHED,s3://kaggle-nakayama/mlflow/7/62eccbddb74f499a...,2024-11-11 11:10:12.434000+00:00,2024-11-11 11:54:29.011000+00:00,1178.4,48.771536,16.4,4.699546,...,lenta,/workspace/dev/process/model/small_data.py,admin,LOCAL,lenta_tlearner,None,causalml,tlearner,a0155cbce46f983ba15dbac44dac5e76078c6225,base_pattern: tlearner training and evaluation...
7,bcde08073c8642dbbe0de077e2e57f98,7,FINISHED,s3://kaggle-nakayama/mlflow/7/bcde08073c8642db...,2024-11-11 10:26:29.546000+00:00,2024-11-11 11:10:11.542000+00:00,1178.4,13.495726,9.5,4.461509,...,lenta,/workspace/dev/process/model/small_data.py,admin,LOCAL,lenta_slearner,None,causalml,slearner,a0155cbce46f983ba15dbac44dac5e76078c6225,base_pattern: slearner training and evaluation...
8,d7af05f9fc3e4f8f954d344a64c647c5,7,FINISHED,s3://kaggle-nakayama/mlflow/7/d7af05f9fc3e4f8f...,2024-11-11 06:35:23.616000+00:00,2024-11-11 10:26:28.643000+00:00,1178.4,22.396624,15.5,20.971774,...,lenta,/workspace/dev/process/model/small_data.py,admin,LOCAL,lenta_rlearner,None,causalml,rlearner,a0155cbce46f983ba15dbac44dac5e76078c6225,base_pattern: rlearner training and evaluation...
9,3061503164874cf3a7b3b65e0e8e72ed,7,FINISHED,s3://kaggle-nakayama/mlflow/7/3061503164874cf3...,2024-11-11 05:22:25.247000+00:00,2024-11-11 06:35:22.710000+00:00,1178.4,-3.423729,9.9,6.716512,...,lenta,/workspace/dev/process/model/small_data.py,admin,LOCAL,lenta_xlearner,None,causalml,xlearner,a0155cbce46f983ba15dbac44dac5e76078c6225,base_pattern: xlearner training and evaluation...
10,a3d1e9d81e384f09b6346aff2e66fdf3,7,FINISHED,s3://kaggle-nakayama/mlflow/7/a3d1e9d81e384f09...,2024-11-11 04:10:01.770000+00:00,2024-11-11 05:22:24.407000+00:00,1178.4,7.843373,14.5,7.564386,...,lenta,/workspace/dev/process/model/small_data.py,admin,LOCAL,lenta_drlearner,None,causalml,drlearner,a0155cbce46f983ba15dbac44dac5e76078c6225,base_pattern: drlearner training and evaluatio...


In [41]:
with mlflow.start_run(run_name=f"{dataset}", tags={"dataset": dataset}) as parent_run:
    pass

2024/11/23 09:43:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run lenta at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/7/runs/958128ce482042c8aea9d2a5474e3b1c.
2024/11/23 09:43:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/7.


In [42]:
for run_id in run_df.run_id:
    run = client.set_tag(run_id, "mlflow.parentRunId", parent_run.info.run_id)